In [ ]:
!pwd

In [ ]:
!python --version

In [ ]:
%cd /content/shap-ePersonalModel/shap_e

In [ ]:
#Run once per session (files get deleted after runtime resource terminates):

!git clone https://github.com/openai/shap-e
%cd shap-e
!pip install -e .

#!pip install pytorch3d


In [1]:
import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
#Check if using gpu 

import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

Num GPUs Available:  1
Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce RTX 3080 Ti, pci bus id: 0000:09:00.0, compute capability: 8.6



In [3]:
xm = load_model('transmitter', device=device)
model = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

  0%|          | 0.00/1.78G [00:00<?, ?iB/s]

100%|███████████████████████████████████████| 890M/890M [01:31<00:00, 10.2MiB/s]


  0%|          | 0.00/1.26G [00:00<?, ?iB/s]

In [6]:
batch_size = 1 #Default 4
guidance_scale = 15.0 #Just like CFG Scale in Stable Diffusion
prompt = input("Enter what you want Shap-e to generate here:")
#prompt = "bird with human face"

latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(texts=[prompt] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

  0%|          | 0/32 [00:00<?, ?it/s]


KeyboardInterrupt



In [ ]:
render_mode = 'stf' # you can change this to 'nerf' or 'stf'
size = 64 # this is the size of the renders; higher values take longer to render.

cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))

In [ ]:
output_path = "./shap-e/"
# Create the Vertex_Colors folder if it doesn't exist
vc_dir = os.path.join(os.path.dirname(output_path), "vertex_colors")
if not os.path.exists(vc_dir):
    os.makedirs(vc_dir)
print(vc_dir)

In [ ]:
cameras = create_pan_cameras(size, device)  
for i, latent in enumerate(latents):  
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)  
    display(gif_widget(images))  
    # Extract color for final mesh   
    for j, image in enumerate(images):  
        filename = os.path.join("vertex_colors", f"color_{i}_{j}.png")  
        image.save(filename)  

In [ ]:
# Example of saving the latents as meshes.
from shap_e.util.notebooks import decode_latent_mesh

for i, latent in enumerate(latents):
    t = decode_latent_mesh(xm, latent).tri_mesh()
    with open(f'example_mesh_{i}.ply', 'wb') as f:
        t.write_ply(f)
    with open(f'example_mesh_{i}.obj', 'w') as f:
        t.write_obj(f)

In [ ]:
%pip install bpy

In [ ]:
#WARNING: Installs Blender for Linux machines. RUN ONLY ON MACHINES THAT DELETE COMPUTE INSTANCES (e.g. Colab)
#!curl -LO https://mirror.clarkson.edu/blender/release/Blender4.0/blender-4.0.2-linux-x64.tar.xz


In [ ]:
#Extracts downloaded Blender 
'''
import tarfile

with tarfile.open('/content/shap-e/blender-4.0.2-linux-x64.tar.xz', 'r:xz') as tar:
    tar.extractall()

'''

In [ ]:
%pwd

In [ ]:
# !/content/shap-e/blender-4.0.2-linux-x64/blender -b -P /content/shap-e/vertexColorexport.py -- /content/shap-e/example_mesh_0.obj


In [ ]:
%python vertexColorImport.py